In [1]:
# -*- coding: utf-8 -*-

#@author: alison

import re
import string
import pickle
import keras
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding
from keras.layers import LSTM, Activation, Input
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
file_list = drive.ListFile({'q': "'1Hx5OP1Yrlh37yYzSMtsv6Ui_fuzOuG04' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: lstm_es_a.ipynb, id: 1yZRPqV6WKQ8OeNwcfpQwj0jqiIMehAHK
title: gru_es_a.ipynb, id: 11DtYTZbYyKHgTFIMXhy8Y4I7c3_0z_W0
title: CNN_en_a.ipynb, id: 19oqoK5aiZrBvFOPN1v89xNcXLevjJY0s
title: WordEmbedding-biLSTM_en_a.ipynb, id: 1JOBvoH52NYgevXOGS-K5Cw4ggR4VYWZP
title: SelfAttention-biLSTM_en_a.ipynb, id: 1wh-yg27fq8OWwO8Jj-Oy47leYf9ujX7U
title: WordEmbedding-LSTM_en_a.ipynb, id: 1S_nrY6BmP5Bd6jTGwGLhhQW_W6ZZK6Tf
title: WordEmbedding-GRU_en_a.ipynb, id: 1fcw8EGX2Hdz-tTbz0NDEhZlOHiPvKNsY
title: bi_lstm_en_a.ipynb, id: 1O-dGg5mtIxPsI4JY16dcp4ytezzN5NQZ
title: lstm_en_a.ipynb, id: 1w3Rq-g3l6u5SsBoXoNcKeciE0YUMLJcP
title: gru_en_a.ipynb, id: 1qHxV1RB5vy_M889397xa1Ep4kEx_9RIa
title: CNN_es_a.ipynb, id: 1UDY2L6Ap8rqV0HtWw9tnLICN4MUD3Nno
title: WordEmbedding-biLSTM_es_a.ipynb, id: 14eSqh0xSpcRJXYwU8Qkv9eJaSW_dt3zb
title: WordEmbedding-LSTM_es_a.ipynb, id: 16FhVQWB5WvbRXDGkDgsLJWOTqyKwaDwb
title: WordEmbedding-GRU_es_a.ipynb, id: 1ahrxlOEINVnCp05yCjmLx8b4v-5SSGL6
title: MachineLearning_en.ipynb

In [0]:
train_downloaded = drive.CreateFile({'id': '18OJPw3-c99gT1XgKZRXCkI_7xexnBV3z'})
train_downloaded.GetContentFile('train_es.tsv')
test_downloaded = drive.CreateFile({'id': '1EC9OkjN6PR5RqehcYXoci7dsuXmlMA0f'})
test_downloaded.GetContentFile('dev_es.tsv')
trial_downloaded = drive.CreateFile({'id': '1lwuVtwkeYCoBn4wsOEwq1Y1gBe4sazaS'})
trial_downloaded.GetContentFile('trial_es.tsv')

In [0]:
train = pd.read_csv('train_es.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('dev_es.tsv', delimiter='\t',encoding='utf-8')
#trial = pd.read_csv('trial_es.tsv', delimiter='\t',encoding='utf-8')

In [0]:
# Etapa de pré-processamento

def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("spanish"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('spanish')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [0]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [0]:
train_text = train['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
y_train    = train['HS']
id_train   = train['id']

test_text  = dev['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
y_test     = dev['HS']
id_test    = dev['id']

data = np.concatenate((train_text, test_text), axis=0)
classes = np.concatenate((y_train, y_test), axis=0)

In [0]:
max_features = 25000
maxlen = 100
batch_size = 100
epochs = 5

# Treina um tokenizaddor nos dados de treino
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data)

# Tokeniza os dados
X = tokenizer.texts_to_sequences(data)
Y = tokenizer.texts_to_sequences(test_text)

tweets = sequence.pad_sequences(X, maxlen=maxlen)
x_test = sequence.pad_sequences(Y, maxlen=maxlen)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

In [22]:
# Fase de classificação de sentimentos

tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(max_features, 256, input_length=maxlen, trainable=True)(tweet_input)

gru1 = LSTM(256, activation='tanh', dropout=0.2, recurrent_dropout=0.1, kernel_initializer='normal', return_sequences=True)(embedding)

gru2 = LSTM(256, activation='tanh')(gru1)

dens = Dense(256, activation='relu')(gru2)

output = Dense(1, activation='sigmoid')(dens)

model = Model(inputs=tweet_input, outputs=output)

#opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.825, epsilon=1e-08)
#opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

y_pred = (model.predict(x_test, batch_size=batch_size, verbose=1) > .5).astype(int)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 256)          6400000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 7,516,673
Trainable params: 7,516,673
Non-trainable params: 0
_________________________________________________________________


In [23]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

print("F1.........: %f" %(f1_score(y_test, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(y_test, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(y_test, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(y_test, y_pred)))

F1.........: 0.881036
Precision..: 0.879949
Recall.....: 0.882996
Accuracy...: 0.882000


In [24]:
output = []
for array in y_pred:
    output.append(array[0])
print(len(output), len(id_test))

500 500


In [0]:
from google.colab import files

with open("es_a.tsv", "w") as file:
    for i in range(len(y_pred)):
        file.write(str(id_test[i]))
        file.write('\t')
        file.write(str(output[i]))
        file.write('\n')

files.download('es_a.tsv')